## Check Prime Number with Serverless Framework

The original example can be found at: https://github.com/rosafilgueira/dispel4py_training_material/blob/master/dispel4py_simple_workflow_collection/Testing_dispel4py.ipynb

This example uses the serverless framework to execute the workflow given above

Prior set-up 

In [ ]:
import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

Import serverless client module along with required dispel4py modules

In [ ]:
from dispel4py.dispel4py.base import *
from dispel4py.dispel4py.workflow_graph import WorkflowGraph
from client import d4pClient,Process
import random

Create three classes of PEs

In [ ]:
class NumberProducer(ProducerPE):
    def __init__(self):
        ProducerPE.__init__(self)
        
    def _process(self, inputs):
        # this PE produces one input
        result= random.randint(1, 1000)
        return result

class IsPrime(IterativePE):
    def __init__(self):
        IterativePE.__init__(self)
    def _process(self, num):
        # this PE consumes one input and produces one output
        self.log("before checking data - %s - is prime or not" % num)
        if all(num % i for i in range(2, num)):
            return num

class PrintPrime(ConsumerPE):
    def __init__(self):
        ConsumerPE.__init__(self)
    def _process(self, num):
        # this PE consumes one input
        self.log("the number %s is prime" % num)

Create PE object from each class and connect together to form a workflow 

In [ ]:
client = d4pClient()

producer = NumberProducer()
isprime = IsPrime()
printprime = PrintPrime()

graph = WorkflowGraph()
graph.connect(producer, 'output', isprime, 'input')
graph.connect(isprime, 'output', printprime, 'input')

#### Optional: Register workflow 

This allows you to access the workflow again in the future.

Note: registering a workflow automatically registers the PEs in the workflow

In [ ]:
workflow_id = client.register_Workflow(graph,"primeGraph")

#### Execute Workflow

Option 1: Execute workflow directly

In [ ]:
result = client.run(graph,input=5)

print(result)

Option 2: Retrieve from registry and execute 

In [ ]:
workflow = client.get_Workflow("primeGraph")
result = client.run(workflow,input=5)
print(result)

#### Execution Options

Simple

In [ ]:
result = client.run(workflow,input=5,process=Process.SIMPLE)
print(result)

Multi 

In [ ]:


from dispel4py.dispel4py.new.multi_process import process as multi_process
from easydict import EasyDict as edict

args = edict({'num':5, 'simple':False})

#client.describe(workflow)

#multi_process(graph, {producer: 5}, args)

#print(type(workflow))

#result = client.run(workflow,input=5,process=Process.MULTI,args={'num':5, 'simple':False})

#print(result)

Dynamic

In [ ]:
args = {'num':5, 'simple':False, 'redis_ip':'localhost', 'redis_port':'6379'}

result = client.run(workflow,input=5,process=Process.DYNAMIC,args=args)

print(result)